# Debugging for Python

<table border="0">
 <tr>
    <td><b style="font-size:48px; text-align: left">History of Bugs</b></td>
    <td><b style="font-size:30px"></b></td>
 </tr>
 <tr>
    <td style='width: 60%'><p style="font-size:34px">It has been just so in all of my inventions. The first step is an intuition, and comes with a burst, then difficulties arise—this thing gives out and [it is] then that "Bugs"—as such little faults and difficulties are called—show themselves and months of intense watching, study and labor are requisite before commercial success or failure is certainly reached. </p>
        <p><b style='font-size:30px'>Thomas Edison, 1878 letter to an associate</b></p></td>
    <td><img src="https://media.giphy.com/media/UAUtB4Oi9U4EM/giphy.gif" width="640" height="480" frameBorder="0" />
        <p></p>.</td>
 </tr>
</table>

### Grace Hopper and The Actual Bug

![Actual Bug](images/BugExample.jpg)

## Debugging Algorithm

* Gather information

* Form a hypothesis

* Test the hypothesis

* Repeat until a hypothesis is proven

* Propose a solution

## Tools
* Try/Except Statements
* Logging
* Python Debugger (PDB)

## Try/Except Statements

![Try Hard](images/try_except.jpg)

### Try and Try Again (Except When You Can't)
Handling exceptions (aka errors) is an important part of creating a robust program/script.

What typically happens when an exception occurs?

Think about a time where an exception occurred in the middle of a code block. What did you want to have happen instead of just stopping?

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x)
# What do we expect to happen if we run this line of code?

* **Try**
 * The program will **try** to run the code contained in the try block and if it hits an exception will check for an appropriately caught exception


* **Except**
 * The program will then run the appropriate **except** block.

* **Else**
 * If no exception was caught it will then run any code in an **else** block.

* **Finally**
 * And then finally the program will run any code in the **finally** block.

In [ ]:
# To catch the error:
try:
    import matplotlib.pyplot as plt
    plt.plot(x)
except:
    print('Variable x is not defined :(')
else:
    print('Successful plotting! :D')
finally:
    print("Code's done!")

What are some potential problems with this approach?


<table border="0">
 <tr>
    <td><b style="font-size:48px; text-align: left">The better approach</b></td>
    <td><b style="font-size:30px"></b></td>
 </tr>
 <tr>
    <td style='width: 60%'><p style="font-size:34px">
<code>try:
    plt.plot(x)
except NameError:
    print('Variable x is not defined')
except ValueError:
    print('Value not plottable')
except et cetera:
    et cetera
</code></p>
    <td><img src="images/error.png" width="640" height="480" frameBorder="0" />
        <p></p>.</td>
 </tr>
</table>


In [ ]:
### Example in practice:
class TwitterStreamer():
    def stream_tweets(self,tag_list,user_list):
        # Authentication stuff
        listener = StdOutListener()
        auth = OAuthHandler(
            credentials.CONSUMER_KEY,credentials.CONSUMER_SECRET)
        auth.set_access_token(
            credentials.ACCESS_TOKEN,credentials.ACCESS_TOKEN_SECRET)
        # Creating a stream object
        stream = Stream(auth,listener)
        start = time.time()
        # Turning the firehose on
        try:
            stream.filter(
                track=tag_list,follow=user_list) # listen in on incoming tweets
        except BaseException as e:
            print(f'Error on stream after {
                time.time()-start} seconds: {e}') # print how long stream lasted
            time.sleep(3) # give it a second (or 3)
            print('Still listening . . .')
            self.stream_tweets(
                tag_list,user_list) # start listening again


<table border="0">
 <tr>
    <td><b style="font-size:48px; text-align: left">The betterer approach</b></td>
    <td><b style="font-size:30px"></b></td>
 </tr>
 <tr>
    <td style='width: 60%'><p style="font-size:34px">
<code>try:
    plt.plot(x)
except NameError:
    print('Variable x is not defined')
except ValueError:
    etc,etc
else:
    plt.savefig()
finally:
    print('Job's done!')
</code></p>
    <td><img src="images/error.png" width="640" height="480" frameBorder="0" />
        <p></p>.</td>
 </tr>
</table>

## But we can still do bettererer
What are some problems with the way errors are tracked in this example?

<p align="center">
  <img src="images/logging.jpg" alt='Logbob'/ style="
      display: block;
      margin-left: auto;
      margin-right: auto;
      width: 50%;">
</p>

## Logging (It's better than bad, it's good!)

In [ ]:
import logging

logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')

## Logging

#### Basic Configuration Options
* **level**: The root logger will be set to the specified severity level.
* **filename**: This specifies the file to log into.
* **filemode**: If filename is given, the file is opened in this mode. The default is a, which appends.
* **format**: Changes the output of the log message (many parameters to play with here).

### You can only set the basicConfig once, here's an example
```
logging.basicConfig(level=logging.INFO, \
            filename='app.log', filemode='w+', \
            format='%(name)s - %(levelname)s - %(message)s')
```

![It's better than bad](images/log_example.jpg)

## Logging Example

Write code below that writes a log file with a critical message containing your favorite band, the level of the message and the current time (this will require setting the basicConfig and a little googling).

e.g:
```
Coheed & Cambria - CRITICAL - 28/Oct/2019 10:35:02
```


In [ ]:
import logging
logging.basicConfig(format='%(message)s - %(levelname)s - %(asctime)s')
logging.critical('Coheed & Cambria \\m/')

In [ ]:
# Example of it used in practice
import logging
import requests
from flask import abort, Flask, request

# Set logging parameters
logging.basicConfig(level=logging.INFO, filename='app.log', filemode='w+', \
                    format='%(name)s - %(levelname)s - %(message)s')

app = Flask(__name__)

@app.route('/rec', methods=['POST'])
def find_games():
    try:
        # Parameter options: game names or mechanic names
        params=dict()
        params['games'] = request.args.get('games')
        params['mechanics'] = request.args.get('mechanics')
        
        # Split request into list of strings
        games = params['games'].split(',') if params['games'] else []
        mechanics = params['mechanics'].split(',') if params['mechanics'] else []
        
        # Log successful request
        logging.info(f'Successful request: \n Games: {games if games else "None"} \n Mechanics: {mechanics if mechanics else "None"}')
    except Exception as e:
        logging.exception(f'Bad request: {request}')
        # same as 
        # logging.error(f'Bad request: {request}', exc_info=True)
        abort(400)

    # Perform recommendation
    data = get_nearest(games,mechanics)
    logging.info(f'Results: {[datum["name"] for datum in data]}')
    line = '\n'
    logging.info(f'Returned: {line}{line.join([str(datum)+line for datum in data])}')

    return {'games': data}

## PDB (Python Debugger)

### Breakpoints
Automatically imports pdb and creates a checkpoint within your code

### Keywords
#### n(ext)
 * Run to the next line of code

```
x_train, x_test, y_train, y_test = 
    train_test_split(features,target)
for model_object in model_list:
    breakpoint()
    model = model_object.fit(x_train,y_train)
    y_pred = model.predict(x_train)
    plt.scatter(y_test, y_pred)
```

#### s(tep)
 * Step into the next possible moment that can be stopped (calling a new function or the next line)

```
x_train, x_test, y_train, y_test = 
train_test_split(features,target)
for model_object in model_list:
    breakpoint()
    model = model_object.fit(x_train,y_train)
    y_pred = model.predict(x_train)
    plt.scatter(y_test, y_pred)
```

#### c(ontinue)
* Run until the next breakpoint is hit

```
x_train, x_test, y_train, y_test = train_test_split(features,target)
for model_object in model_list:
    breakpoint()
    model = model_object.fit(x_train,y_train)
    y_pred = model.predict(x_train)
    plt.scatter(y_test, y_pred)
```

#### l(ist)
* Print out the context of the code line you are in

#### exit
* Yeet out of the debugger

# For Loop Example

In [ ]:
from IPython.display import clear_output as bye
import Ledger
bill = 100.0
breakpoint()
for year in range(1,4):
    bill = Ledger.add_interest(bill)

#### b(reak) [ ([filename:]lineno | function) [, condition] ]
* b (break) lists all of the breakpoints
* b followed by the above conditional sets a new breakpoint

In [ ]:
from IPython.display import clear_output as bye
import Ledger
bill = 100.0
breakpoint()
for year in range(1,4):
    bill = Ledger.add_interest(bill)

## Dependence Example

In [ ]:
import Ledger

In [ ]:
Ledger.get_bill()

In [ ]:
bills=[]
for i in range(4):
    bills.append(Ledger.get_bill())

In [ ]:
Ledger.get_paid(bills)

In [ ]:
breakpoint()
Ledger.get_paid(bills)

## Problem:
Explore the following code using the breakpoint and figure out what went wrong in the loops:
1. Set a breakpoint for when Ledger.get_bill gets called 
    * Remember jupyter notebooks have a lot of hidden functions running on the stack
2. C(ontinue) into the function, check your context using l(ist) and run any commands to check variables
3. **n**(ext), **s**(tep) and **c**(ontinue) are your best friends

In [ ]:
import Ledger
bills=[]
for i in range(1,5):
    breakpoint()
    bills.append(Ledger.get_bill())

## Resources
https://www.slideshare.net/svilen.ivanov/the-art-of-debugging

https://www.cse.unr.edu/~bebis/CS308/PowerPoint/DEBUGGING.ppt

https://www.w3schools.com/python/python_try_except.asp

https://realpython.com/python-logging/

https://paxson.io/python-logging/ (email logs)

## Code Examples
Logging Flask server https://github.com/danjizquierdo/BoardGameRecommender/blob/master/recommender.py

Try/Except Twitter stream
https://github.com/danjizquierdo/1stPrimaryDebateNight2020/blob/master/twitter_migrator.py